In [92]:
import cvzone
from cvzone.HandTrackingModule import HandDetector
import pycaw

import cv2
import numpy as np
import time
import mediapipe 
from cvzone.FPS import FPS
import os
import pyautogui as control
control.FAILSAFE =False
import math
import csv
import time
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume#
#################imort volum Liberary#######

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = interface.QueryInterface(IAudioEndpointVolume)
volume.GetMute()
volume.GetMasterVolumeLevel()
volume.GetVolumeRange()
volume.SetMasterVolumeLevel(vol, None)
############# our Variables ############
wCam, hCam = 640, 480
cap = cv2.VideoCapture(0)
cap.set(3,wCam)
cap.set(4,hCam)
offset = 20
imgSize = 300
counter = 0
offset  = 20
vol = 0
newVolumeBar = 300
volumRange = volume.GetVolumeRange()
minVol = volumRange[0]
maxVol = volumRange[1]
############################### 
detector = HandDetector(staticMode=False,maxHands=2,modelComplexity=1,detectionCon=0.5, minTrackCon=0.5)   
############################################
while True :
    _,frame = cap.read()
    frame = cv2.flip(frame,1)
    hands , _ = detector.findHands(frame,draw=False,flipType=False)
    #chack hands
    if hands :
        hand = hands[0]
        x,y,w,h = hand['bbox']
        HandImage = frame[y-offset:y+h+offset,x-offset:x+w+offset]
        lmlist = hand['lmList']
        distance , info,_ = detector.findDistance(lmlist[4][0:2],lmlist[8][0:2],color=(0,255,0))
        #map volum range to hand range 
        #volum range =(-95.25, 0.0, 0.75)
        #hand Range = 120 ,8
        newVolume   = np.interp(distance,[50,120],[minVol,maxVol])
        newVolumeBar= np.interp(distance,[50,120],[400,150])
        #change vloume value
        volume.SetMasterVolumeLevel(newVolume, None)
    cv2.rectangle(frame,(20,150),(50,400),(255,0,0),3)
    cv2.rectangle(frame,(20,int(newVolumeBar)),(50,400),(0,255,0),cv2.FILLED)
    imgPNG = cv2.imread("C:/Users/Geka/Desktop/update.png",cv2.IMREAD_UNCHANGED)
    imgOverlay = cvzone.overlayPNG(frame, imgPNG, pos=[3, 430])
    cv2.imshow('Volume Control',frame)
    # Display the camera feed
    key = cv2.waitKey(10)
    if key == ord('q'):
        break
cap.release()
cv2.destroyAllWindows() 
